In [1]:
import pandas as pd
import db
import json
from deep_translator import GoogleTranslator
from deep_translator import (GoogleTranslator,
                             batch_detection)

In [58]:
sql = """
SELECT ROWID, *
FROM Song
"""

df = db.dataframe(sql)
df.head()

,rowid,artist,title,year
0,1,1,Dreadlock Holiday,1978
1,2,1,I'm Not In Love (Albumversie),1975
2,3,1,The Wall Street Shuffle,1974
3,4,1,I'm Mandy Fly Me,1976
4,5,1,Donna,1973


In [59]:
# load the existing data
# Opening JSON file
f = open('staged/cached_translations.json')

# returns JSON object as 
# a dictionary
output = json.load(f)
f.close()

In [60]:
titles = []
for row in db.raw_sql(sql):
    title = row[2]
    if title not in output and not title.isnumeric():
        titles.append(title)
print('There are %d titles to translate' % len(titles))

There are 668 titles to translate


In [61]:
if len(titles) > 0:
    translations = GoogleTranslator(source='auto', target='en').translate_batch(titles)
    for i in range(len(titles)):
        title = titles[i]
        translation = translations[i]
        language = 'en' if title==translation else 'other'
        output[title] = {
            "language": language,
            "translation": translation
        }


In [62]:
others = {k:v for k, v in output.items() if v['language']=='other'}

keyset = list(others.keys())
print(keyset)

['Iedereen Is Van De Wereld', 'Blauw', 'This Charming Man  ', "Baba O'Riley", '24 Rozen', "Mien, Waar Is M'n Feestneus", 'Stiekem Gedanst', 'Mushanga', 'Pamela (Albumversie)', "L'Italiano", 'Gente Di Mare', 'De Zee', 'Nu Dat Jij Er Bent', 'Met Hart en Ziel', 'Vanmorgen Vloog Ze Nog', 'Wer Bisto', 'Rendez-Vous 6:02', 'Ti Amo', 'Het Kleine Cafe Aan De Haven', 'Stil In Mij', 'Laat Het Los', 'Wanhoop Niet', 'Meer Dan Een Ander', 'Ik Wou Dat Ik Jou Was', 'Apres Toi', 'Een Kopje Koffie', 'Hou Me Vast', 'Alane', 'Ik Ben Zo Eenzaam Zonder Jou', 'Telkens Weer', 'Samen Zijn', 'Spiegelbeeld', 'De Glimlach Van Een Kind', 'Het Dorp', 'Flappie', 'Durgerdam Slaapt', 'Il Volo', 'Senza Una Donna', 'Cosi Celeste', 'Ik Heb Genoeg Van Jou', 'La Comparsa', 'La Grange', 'Parijs', 'Hemel Valt', 'Zandloper', 'Freestyler', 'Killing Moon ', 'Stan (Albumversie)', "L'amour Toujours", 'Het Land Van', 'Buddy Holly  ', 'Reflektor', 'Treur Niet (Ode Aan Het Leven)', 'Zoutelande', 'Despacito', 'Iederene Hef Een Reden'

In [63]:

lang = batch_detection(keyset, api_key='8377a84551921b28dd1130147e8dc025')
print(lang) # output: [fr, en]


['nl', 'nl', 'en', 'en', 'tr', 'nl', 'nl', 'no', 'nl', 'it', 'pt', 'en', 'nl', 'nl', 'nl', 'pl', 'fr', 'es', 'nl', 'sl', 'nl', 'nl', 'af', 'nl', 'pt', 'nl', 'nl', 'pt', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'da', 'nl', 'it', 'it', 'pt', 'nl', 'es', 'en', 'nl', 'af', 'nl', 'en', 'en', 'nl', 'fr', 'no', 'en', 'de', 'nl', 'nl', 'en', 'nl', 'af', 'nl', 'nl', 'de', 'fr', 'de', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'af', 'nl', 'nl', 'nl', 'af', 'af', 'nl', 'cs', 'nl', 'nl', 'nl', 'es', 'es', 'cs', 'nl', 'nl', 'en', 'it', 'nl', 'da', 'en', 'nl', 'nl', 'nl', 'fy', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl', 'nl']


In [64]:
for i in range(len(keyset)):
    key = keyset[i]
    language = lang[i]
    output[key]['language'] = language

In [65]:
json_object = json.dumps(output, indent = 2)
with open("staged/cached_translations.json", "w") as outfile:
    outfile.write(json_object)
